#### Two-Stage Evaluation

1. Objective Issue Detection : src.detect

2. Subjective Attribute Evaluation: src.evalute

3. Simulate conversation: src.simulate 

In [1]:
from utils import collect_ooc_response
collect_ooc_response() 

In [12]:
# Now we build the evaluator -- Comparative Score (Base on comparing with the buckets)
from src.eval import AOEval
from src.utils import load_requirements

requirements = load_requirements()
aoeval = AOEval(requirements) # Async call enabled (Watch-out for request limit)

In [4]:
import asyncio 

async def run_eval(conversation):
    result = await aoeval.direct_eval(conversation)
    return result

conversation = "haha"
result = await run_eval(conversation)
print(result)

{'Funny': 'Ok', 'Concise': 'Ok'}
